# ResNet

https://docs.pytorch.org/vision/main/models/resnet.html

### SVHN Single View House Numbers

http://ufldl.stanford.edu/housenumbers/

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 10
batch_size = 64
num_epochs = 3
learning_rate = 1e-3

### 데이터 로드

In [3]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4377, 0.4438, 0.4728], # 이 데이터셋의 계산된 평균/표준편차
                         std=[0.1980, 0.2010, 0.1970])
])

train_dataset = datasets.SVHN(root='./', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./', split='test', download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# 원본이미지
image = train_dataset.data[0]
label = train_dataset.labels[0]

print(image.shape)
print(label)

(3, 32, 32)
1


In [5]:
# transform 적용
image, label = train_dataset[0] 

print(image.shape)
print(label)

torch.Size([3, 224, 224])
1


### 모델 로드

In [6]:
from torchsummary import summary

model = models.resnet18(weights=True)
summary(model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

c:\Users\Playdata\anaconda3\envs\llm_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# 사전학습 모델 가중치 고정 및 출력층 변경
for param in model.parameters():
    param.requires_grad = False # freezing

in_features = model.fc.in_features # 512
model.fc = nn.Linear(in_features, num_classes)

summary(model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

### 학습

In [8]:
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
params_to_update = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(params_to_update, lr=learning_rate)

model = model.to(device)

def train_epoch(model, loader):
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)
        _, predicted = outputs.max(1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    return train_loss / train_total, train_correct / train_total

def eval_epoch(model, loader):
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * imgs.size(0)
        _, predicted = outputs.max(1)
        val_correct += (predicted == labels).sum().item()
        val_total += labels.size(0)

    return val_loss / val_total, val_correct / val_total

for epoch in tqdm(range(num_epochs)):
    train_loss, train_acc = train_epoch(model, train_loader)
    val_loss, val_acc = eval_epoch(model, test_loader)
    print(f'Epoch { epoch + 1}/{num_epochs}: {train_loss=:.4f} {train_acc=:.4f} {val_loss=:.4f} {val_acc=:.4f}')

 33%|███▎      | 1/3 [26:43<53:27, 1603.94s/it]

Epoch 1/3: train_loss=1.8018 train_acc=0.3834 val_loss=1.6518 val_acc=0.4366


 67%|██████▋   | 2/3 [53:55<27:00, 1620.17s/it]

Epoch 2/3: train_loss=1.6474 train_acc=0.4413 val_loss=1.6063 val_acc=0.4556


100%|██████████| 3/3 [1:20:57<00:00, 1619.03s/it]

Epoch 3/3: train_loss=1.6172 train_acc=0.4515 val_loss=1.5767 val_acc=0.4640
